In [1]:
import os
import re

import pandas as pd

from collections import Counter
from orv_cleanup_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
data_dir = '/home/common/regulation_data/parsed_reports/'
main_df = pd.read_csv(data_dir + 'main_df.csv')

otm_tables_fnames = sorted([fn for fn in os.listdir(data_dir) if not (fn.startswith('main') or fn.startswith('.'))])
otm_tables = {fn[:-4]: pd.read_csv(data_dir + fn) for fn in otm_tables_fnames}
otm_tables.keys()

dict_keys(['0_business', '1_business', '2_business', 'cancel_duties', 'expenses', 'goals', 'group_changes', 'group_expenses', 'groups', 'kpi', 'neccessary_measures', 'new_functions', 'notification_info', 'public_discussion', 'risks'])

### Тестирование функций очистки

In [5]:
test = pd.Series([
    '', '\r', 'dfd\n', '\r\n', '\n\r\n', 
    'u', '.',
    '-', '- --', 
    '____', 
    'nan', 'нет',
    '(место для текстового описания)',
    '-(место для текстового описания)'
])
test.apply(clean_up)

0     $#*!
1     $#*!
2     $#*!
3     $#*!
4     $#*!
5     $#*!
6     $#*!
7     $#*!
8     $#*!
9     $#*!
10    $#*!
11    $#*!
12    $#*!
13    $#*!
dtype: object

In [6]:
# Поиск самых частых значений
def most_freq(series, num):
    return sorted(Counter(series).items(), key=lambda item: (-item[1], item[0]))[:num]


# Поиск самых коротких заполнений
def shortest(series, num):
    return sorted(Counter(series).items(), key=lambda item: (len(item[0]), item[1]))[:num]


def get_column_features(df, column, top_freq):
    cleaned_field = df[column].apply(clean_up)
    mf = pd.DataFrame(most_freq(cleaned_field, top_freq))
    sh = pd.DataFrame(shortest(cleaned_field, top_freq))
    return mf, sh

### К каким полям будем применять функции очистки (ключи будут использованы как названия листов в Excel)

In [7]:
goals_df = otm_tables['goals']
groups_df = otm_tables['groups']
expenses_df = otm_tables['expenses']
group_changes_df = otm_tables['group_changes']
group_expenses_df = otm_tables['group_expenses']
risks_df = otm_tables['risks']
necessary_measures_df = otm_tables['neccessary_measures']
cancel_duties_df  = otm_tables['cancel_duties']
new_functions_df = otm_tables['new_functions']

check_fields = {
    'general_info@problem': (main_df, 'general_info: Краткое описание проблемы, на решение которой направлен предлагаемый способ регулирования'),
    'general_info@goals': (main_df, 'general_info: Краткое описание целей предлагаемого регулирования'),
    'general_info@way_of_reg': (main_df, 'general_info: Краткое описание предлагаемого способа регулирования'),
    'degree@justify': (main_df, 'degree: Обоснование отнесения проекта акта к определенной степени регулирующего воздействия'),
    'problem@description': (main_df, 'problem: Описание проблемы, на решение которой направлен предлагаемый способ регулирования, условий и факторов ее существования'),
    'problem@negative': (main_df, 'problem: Негативные эффекты, возникающие в связи с наличием проблемы'),
    'problem@origin': (main_df, 'problem: Информация о возникновении, выявлении проблемы, принятых мерах, направленных на ее решение, а также затраченных ресурсах и достигнутых результатах решения проблемы'),
    'problem@conditions': (main_df, 'problem: Описание условий, при которых проблема может быть решена в целом без вмешательства со стороны государства'),
    'problem@sources': (main_df, 'problem: Источники данных'),
    'int_exp@experience': (main_df, 'int_exp: Международный опыт в соответствующих сферах деятельности'),
    'int_exp@sources': (main_df, 'int_exp: Источники данных'),
    'goals@goals': (goals_df, 'Цели предлагаемого регулирования'),
    'goals@timing': (goals_df, 'Установленные сроки достижения целей предлагаемого регулирования'),
    'goals@conform': (main_df, 'goals: Обоснование соответствия целей предлагаемого регулирования принципам правового регулирования, программным документам Президента Российской Федерации и Правительства Российской Федерации'),
    'description@way': (main_df, 'description: Описание предлагаемого способа решения проблемы и преодоления связанных с ней негативных эффектов'),
    'description@other': (main_df, 'description: Описание иных способов решения проблемы (с указанием того, каким образом каждым из способов могла бы быть решена проблема)'),
    'description@why': (main_df, 'description: Обоснование выбора предлагаемого способа решения проблемы'),
    'groups@sources': (main_df, 'groups: Источники данных'),
    'groups@groups': (groups_df, 'Группа участников отношений'),
    'groups@count': (groups_df, 'Оценка количества участников отношений'),
    # expenses должны быть здесь, а не в конце списка
    'group_changes@groups': (group_changes_df, 'Группа участников отношений'),
    'group_changes@new_func': (group_changes_df, 'Описание новых преимуществ, обязанностей, ограничений или изменения содержания существующих обязанностей и ограничений'),
    'group_changes@order': (group_changes_df, 'Порядок организации исполнения обязанностей и ограничений'),
    'group_expenses@sources': (main_df, 'group_expenses: Источники данных'),
    'group_expenses@groups': (group_expenses_df, 'Группа участников отношений'),
    'group_expenses@new_func': (group_expenses_df, 'Описание новых или изменения содержания существующих обязанностей и ограничений'),
    'group_expenses@expenses': (group_expenses_df, 'Описание и оценка видов расходов (доходов)'),
    'risks@sources': (main_df, 'risks: Источники данных'),
    'risks@risks': (risks_df, 'Риски решения проблемы предложенным способом и риски негативных последствий'),
    'risks@proba': (risks_df, 'Оценки вероятности наступления рисков'),
    'risks@control': (risks_df, 'Методы контроля эффективности избранного способа достижения целей регулирования'),
    'risks@degree': (risks_df, 'Степень контроля рисков'),
    'business@sources_structure': (main_df, 'business: Источники данных (оценка структуры)'),
    'business@msp_impact': (main_df, 'business: Оценка влияния проекта акта на достижение целевых ориентиров Стратегии развития малого и среднего предпринимательства в Российской Федерации'),
    'business@sources_impact': (main_df, 'business: Источники данных (оценка влияния)'),
    'business@npa': (main_df, 'business: Нормативно-правовые меры, предпринятые для сокращения диспропорций в нагрузке'),
    'business@org': (main_df, 'business: Организационные меры, предпринятые для сокращения диспропорций в нагрузке'),
    'neccessary_measures@total': (main_df, 'neccessary_measures: Общий объем затрат на необходимые для достижения заявленных целей регулирования организационно-технические, методологические, информационные и иные мероприятия (млн. руб.)'),
    'necessary_measures@events': (necessary_measures_df, 'Мероприятия, необходимые для достижения целей регулирования'),
    'necessary_measures@timing': (necessary_measures_df, 'Сроки мероприятий'),
    'necessary_measures@descr': (necessary_measures_df, 'Описание ожидаемого результата'),
    'necessary_measures@finance': (necessary_measures_df, 'Объем финансирования'),
    'necessary_measures@finance_source': (necessary_measures_df, 'Источники финансирования'),
    'kpi@info': (main_df, 'kpi: Информация о программах мониторинга и иных способах (методах) оценки достижения заявленных целей регулирования'),
    'kpi@money': (main_df, 'kpi: Оценка затрат на осуществление мониторинга (в среднем в год, млн. руб.)'),
    'kpi@sources': (main_df, 'kpi: Описание источников информации для расчета показателей (индикаторов)'),
    'notification_info@persons': (main_df, 'notification_info: Сведения о лицах, предоставивших предложения'),
    'anticorr_expertise@experts': (main_df, 'anticorr_expertise: Указать (при наличии) количество поступивших заключений от независимых экспертов (шт.)'),
    'anticorr_expertise@corruption': (main_df, 'anticorr_expertise: Выявленные коррупциогенные факторы и их способы устранения'),
    'public_discussion@foiv': (main_df, 'public_discussion: Сведения о федеральных органах исполнительной власти и представителях предпринимательского сообщества, извещенных о проведении публичных консультаций'),
    'public_discussion@persons': (main_df, 'public_discussion: Сведения о лицах, представивших предложения'),
    'cancel_duties@npa': (main_df, 'cancel_duties: Нормативный правовой акт, в котором содержатся отменяемые обязанности, запреты или ограничения'),
    'cancel_duties@canceled': (cancel_duties_df, 'Описание отменяемых обязанностей, запретов или ограничений'),
    'cancel_duties@money': (cancel_duties_df, 'Описание и оценка затрат на выполнение отменяемых обязанностей, запретов или ограничений'),
    'new_functions@descr': (new_functions_df, 'Описание новых или изменения существующих функций, полномочий, обязанностей или прав'),
    'new_functions@order': (new_functions_df, 'Порядок реализации'),
    'new_functions@money': (new_functions_df, 'Оценка изменения трудозатрат и (или) потребностей в иных ресурсах'),
    'new_functions@name': (new_functions_df, 'Наименование органа'),
#     'expenses@new_func': (expenses_df, 'Наименование новой или изменяемой функции, полномочия, обязанности или права'),
#     'expenses@organ': (expenses_df, 'Наименование органа'),
#     'expenses@onetime': (expenses_df, 'Единовременные расходы в год возникновения'),
#     'expenses@periodic': (expenses_df, 'Периодические расходы за период'),
#     'expenses@income': (expenses_df, 'Возможные поступления за период'),
#     'expenses@other_info': (main_df, 'expenses: Иные сведения о расходах (возможных поступлениях) бюджетов бюджетной системы Российской Федерации'),
#     'expenses@sources': (main_df, 'expenses: Источники данных')
}

In [8]:
top_freq = 50

writer = pd.ExcelWriter('fillings_sample.xlsx')

header = ['Заполнение', 'Количество']

In [9]:
# pd.DataFrame.from_dict(check_fields, orient='index').to_excel(writer, sheet_name='info', header=False)    
for k in check_fields.keys():
    df = check_fields.get(k)[0]
    field_name = check_fields.get(k)[1]
    print(f'Processing field: \n\t{field_name}')
    cleaned_field = df[field_name].apply(clean_up)
    
    mf = pd.DataFrame(most_freq(cleaned_field, top_freq))
    sh = pd.DataFrame(shortest(cleaned_field, top_freq))

    mf.to_excel(writer, sheet_name=k+'_freq', index=False, header=header)    
    sh.to_excel(writer, sheet_name=k+'_short', index=False, header=header)
writer.save()

Processing field: 
	general_info: Краткое описание проблемы, на решение которой направлен предлагаемый способ регулирования
Processing field: 
	general_info: Краткое описание целей предлагаемого регулирования
Processing field: 
	general_info: Краткое описание предлагаемого способа регулирования
Processing field: 
	degree: Обоснование отнесения проекта акта к определенной степени регулирующего воздействия
Processing field: 
	problem: Описание проблемы, на решение которой направлен предлагаемый способ регулирования, условий и факторов ее существования
Processing field: 
	problem: Негативные эффекты, возникающие в связи с наличием проблемы
Processing field: 
	problem: Информация о возникновении, выявлении проблемы, принятых мерах, направленных на ее решение, а также затраченных ресурсах и достигнутых результатах решения проблемы
Processing field: 
	problem: Описание условий, при которых проблема может быть решена в целом без вмешательства со стороны государства
Processing field: 
	problem

/home/zorins/PycharmProjects/cpur/venv/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing field: 
	Источники финансирования
Processing field: 
	kpi: Информация о программах мониторинга и иных способах (методах) оценки достижения заявленных целей регулирования
Processing field: 
	kpi: Оценка затрат на осуществление мониторинга (в среднем в год, млн. руб.)
Processing field: 
	kpi: Описание источников информации для расчета показателей (индикаторов)
Processing field: 
	notification_info: Сведения о лицах, предоставивших предложения
Processing field: 
	anticorr_expertise: Указать (при наличии) количество поступивших заключений от независимых экспертов (шт.)
Processing field: 
	anticorr_expertise: Выявленные коррупциогенные факторы и их способы устранения
Processing field: 
	public_discussion: Сведения о федеральных органах исполнительной власти и представителях предпринимательского сообщества, извещенных о проведении публичных консультаций
Processing field: 
	public_discussion: Сведения о лицах, представивших предложения
Processing field: 
	cancel_duties: Нормативный